# 0X Notebook-Template

In [1]:
import sys

sys.path.append("../../")

import pandas as pd
import connection as connsettings

from pyspark.sql import functions as f

from etl.etl import HadoopStdETL
from etl.clients import HiveClient, HDFileSystemClient
from etl.datamodels import HostDataClass, TableDataClass
from etl.enums import eHdfsFileType
from etl.connectors import CsvConnector
from datetime import datetime

In [2]:
conn = CsvConnector(path="./database/datev.dbo.client.csv", sep="|")
conn

CsvConnector(
	'path=./database/datev.dbo.client.csv',
	'sep=|'
)

In [3]:
for i, batch in conn.iterbatches(batchsize=3):
    display(batch)

,id,name,address,change_field
0,57,Omicron Systems GmbH,"Musterstraße 57, 10163 Berlin",cf uc: 1
1,19,Epic Solutions SE,"Musterstraße 19, 10125 Berlin",cf
2,66,Quantum Consulting SE,"Musterstraße 66, 10172 Berlin",cf uc: 1


,id,name,address,change_field
3,30,Harmony Services GmbH,"Musterstraße 30, 10136 Berlin",cf
4,17,Echo Solutions SE,"Musterstraße 17, 10123 Berlin",cf
5,50,Mu Enterprises AG,"Musterstraße 50, 10156 Berlin",cf


,id,name,address,change_field
6,49,Momentum Innovations SE,"Musterstraße 49, 10155 Berlin",cf
7,3,Apex Corporation GmbH,"Musterstraße 3, 10109 Berlin",cf
8,88,Vortex Technologies SE,"Musterstraße 88, 10194 Berlin",cf uc: 1


,id,name,address,change_field
9,55,Oasis Corporation Ltd.,"Musterstraße 55, 10161 Berlin",cf


In [4]:
hive_host = HostDataClass(host=connsettings.HIVE_HOST, port=connsettings.HIVE_PORT)
hdfs_host = HostDataClass(host=connsettings.HDFS_HOST, port=connsettings.HDFS_PORT)
hdfs_client = HDFileSystemClient(hdfs_host=hdfs_host, hdfs_username="enricogoerlitz")
hive_client = HiveClient(host=hive_host, thrift_port=connsettings.HIVE_THRIFT_PORT)

In [13]:
conn = CsvConnector(path="./database/datev.dbo.client.csv", sep="|")

etl = HadoopStdETL(
    conn=conn,
    hdfs_client=hdfs_client,
    hive_client=hive_client,
    tmp_path="/edw/hive/tmp/",
    bck_path="/edw/hive/bck_psa/",
    dist_path="/edw/hive/psa/",
    table=TableDataClass(
        database="datev",
        schema="dbo",
        table_name="client",
        pk=["id"]
    ),
    use_spark=True,
    change_columns=["name", "change_field"],
    historize=True
)

etl.start(batchsize=3)

START LOAD TO TMP
UPDATE /edw/hive/psa/datev/dbo/client/client
UPDATED DF:


,pk,guid,id,name,address,change_field,row_is_current,row_valid_to,row_is_deleted,row_filepath,row_filepath_tmp,is_batch_file,COLDIFF_KEY,guid_tmp,COLDIFF_KEY_TMP


DELETED DF:


,pk,guid,id,name,address,change_field,row_is_current,row_valid_to,row_is_deleted,row_filepath,row_filepath_tmp,is_batch_file,COLDIFF_KEY,guid_tmp,COLDIFF_KEY_TMP
0,88,8586908f-0ff8-4fec-957c-8102d0972b07,88,Vortex Technologies SE,"Musterstraße 88, 10194 Berlin",cf uc: 2,1,2100-12-31 00:00:00,0,/edw/hive/psa/datev/dbo/client/client/BATCH_2....,/edw/hive/tmp/datev/dbo/client/BATCH_2.parquet,1,Vortex Technologies SE_cf uc: 2,None,None


TRANSFORMED DF:


,guid,pk,id,name,address,change_field,row_is_current,row_valid_to,row_is_deleted,row_filepath,row_filepath_tmp,is_batch_file
0,581fef1a-b1e5-4f99-b7c3-17c4474fa54d,66,66,Quantum Consulting SE,"Musterstraße 66, 10172 Berlin",cf,0,2024-01-04 00:00:00,0,/edw/hive/psa/datev/dbo/client/client/UPDATE_0...,/edw/hive/psa/datev/dbo/client/client/UPDATE_0...,0
1,8a83f545-c371-4579-8e02-4ca7df52da62,50,50,Mu Enterprises AG,"Musterstraße 50, 10156 Berlin",cf,0,2024-01-04 00:00:00,0,/edw/hive/psa/datev/dbo/client/client/UPDATE_0...,/edw/hive/psa/datev/dbo/client/client/UPDATE_0...,0
2,6d16f3d8-2ef1-460d-a36d-507fa91b0d4c,55,55,Oasis Corporation Ltd.,"Musterstraße 55, 10161 Berlin",cf,0,2024-01-04 00:00:00,0,/edw/hive/psa/datev/dbo/client/client/UPDATE_0...,/edw/hive/psa/datev/dbo/client/client/UPDATE_0...,0
3,818b728c-ffee-43b6-b295-503b181cdc5d,17,17,Echo Solutions SE,"Musterstraße 17, 10123 Berlin",cf,0,2024-01-04 00:00:00,1,/edw/hive/psa/datev/dbo/client/client/UPDATE_0...,/edw/hive/psa/datev/dbo/client/client/UPDATE_0...,0
4,d56a8fc6-bd1e-4304-954c-aa9dab8e016c,19,19,Epic Solutions SE,"Musterstraße 19, 10125 Berlin",cf,0,2024-01-04 00:00:00,0,/edw/hive/psa/datev/dbo/client/client/UPDATE_0...,/edw/hive/psa/datev/dbo/client/client/UPDATE_0...,0
5,2d30b0a6-2a61-471a-9323-87c3e68393d3,88,88,Vortex Technologies SE,"Musterstraße 88, 10194 Berlin",cf,0,2024-01-04 00:00:00,0,/edw/hive/psa/datev/dbo/client/client/UPDATE_0...,/edw/hive/psa/datev/dbo/client/client/UPDATE_0...,0
6,b0d6b07f-28af-4129-bd48-6386ff002557,88,88,Vortex Technologies SE,"Musterstraße 88, 10194 Berlin",cf uc: 1,0,2024-01-04 00:00:00,0,/edw/hive/psa/datev/dbo/client/client/UPDATE_0...,/edw/hive/psa/datev/dbo/client/client/UPDATE_0...,0
0,8586908f-0ff8-4fec-957c-8102d0972b07,88,88,Vortex Technologies SE,"Musterstraße 88, 10194 Berlin",cf uc: 2,0,2024-01-04 00:00:00,1,/edw/hive/psa/datev/dbo/client/client/UPDATE_0...,/edw/hive/psa/datev/dbo/client/client/UPDATE_0...,0


MOVE PSA TO BCK
MOVE TMP TO PSA
DELETE BCK


# Big Test

In [9]:
conn = CsvConnector(path="./database/datev.dbo.employee_pay.csv", sep="|")

etl = HadoopStdETL(
    conn=conn,
    hdfs_client=hdfs_client,
    hive_client=hive_client,
    tmp_path="/edw/hive/tmp/",
    bck_path="/edw/hive/bck_psa/",
    dist_path="/edw/hive/psa/",
    table=TableDataClass(
        database="datev",
        schema="dbo",
        table_name="employee_pay",
        pk=["transaction_date", "employee_id",
            "client_id", "costcenter_id", "paytype_id",
            "amount"]
    ),
    use_spark=True,
    change_columns=None,
    historize=False
)

etl.start(batchsize=10_000)

MOVE PSA TO BCK
MOVE TMP TO PSA
DELETE BCK


In [5]:
conn = CsvConnector(path="./database/datev.dbo.employee_pay.csv", sep="|")

dummy_cols = ["transaction_date", "employee_id",
            "client_id", "costcenter_id", "paytype_id",
            "amount"]
etl = HadoopStdETL(
    conn=conn,
    hdfs_client=hdfs_client,
    hive_client=hive_client,
    tmp_path="/edw/hive/tmp/",
    bck_path="/edw/hive/bck_psa/",
    dist_path="/edw/hive/psa/",
    table=TableDataClass(
        database="datev",
        schema="dbo",
        table_name="employee_pay_hist",
        pk=dummy_cols
    ),
    use_spark=True,
    change_columns=dummy_cols,
    historize=True
)

etl.start(batchsize=10_000)

START LOAD TO TMP
UPDATE /edw/hive/psa/datev/dbo/employee_pay_hist/employee_pay_hist


24/01/04 19:01:02 WARN Utils: Your hostname, MacBook-Air-von-Enrico.local resolves to a loopback address: 127.0.0.1; using 192.168.0.4 instead (on interface en0)
24/01/04 19:01:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/04 19:01:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/01/04 19:01:05 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


UPDATED DF:


,pk,guid,transaction_date,employee_id,client_id,costcenter_id,paytype_id,amount,bdcolumn1,bdcolumn2,...,bdcolumn150,row_is_current,row_valid_to,row_is_deleted,row_filepath,row_filepath_tmp,is_batch_file,COLDIFF_KEY,guid_tmp,COLDIFF_KEY_TMP


DELETED DF:


,pk,guid,transaction_date,employee_id,client_id,costcenter_id,paytype_id,amount,bdcolumn1,bdcolumn2,...,bdcolumn150,row_is_current,row_valid_to,row_is_deleted,row_filepath,row_filepath_tmp,is_batch_file,COLDIFF_KEY,guid_tmp,COLDIFF_KEY_TMP


TRANSFORMED DF:


None

MOVE PSA TO BCK
MOVE TMP TO PSA
DELETE BCK


# DEV

In [13]:
hdfs_client.read_parquet("/edw/hive/psa/datev/dbo/client/client/UPDATE_0.parquet")

,pk,guid,id,name,address,change_field,row_is_current,row_valid_to,row_is_deleted,row_filepath,row_filepath_tmp,is_batch_file,COLDIFF_KEY,guid_tmp,COLDIFF_KEY_TMP
0,91,828cf089-6c36-4ae3-baaf-9e6a53fee5c3,91,Wavelength Ventures AG,"Musterstraße 91, 10197 Berlin",cf uc: 2,0,2024-01-04 00:00:00,0,/edw/hive/psa/datev/dbo/client/client/UPDATE_0...,/edw/hive/psa/datev/dbo/client/client/UPDATE_0...,0,Wavelength Ventures AG_cf uc: 2,b343c776-db21-4f15-90b8-da11f979f204,Wavelength Ventures AG_cf uc: 3
1,49,d4b7e05c-aa5a-43c1-ae2d-d4bf1237d7af,49,Momentum Innovations SE,"Musterstraße 49, 10155 Berlin",cf uc: 2,0,2024-01-04 00:00:00,0,/edw/hive/psa/datev/dbo/client/client/UPDATE_0...,/edw/hive/psa/datev/dbo/client/client/UPDATE_0...,0,Momentum Innovations SE_cf uc: 2,9ff192b3-ec11-4017-8ff4-6875ab4d62f6,Momentum Innovations SE_cf uc: 3


In [16]:
hive_host = HostDataClass(host=connsettings.HIVE_HOST, port=connsettings.HIVE_PORT)
hdfs_host = HostDataClass(host=connsettings.HDFS_HOST, port=connsettings.HDFS_PORT)
hdfs_client = HDFileSystemClient(hdfs_host=hdfs_host, hdfs_username="enricogoerlitz")
hive_client = HiveClient(host=hive_host, thrift_port=connsettings.HIVE_THRIFT_PORT)

In [6]:
tmp_table = TableDataClass("tmp_client", "datev")
table = TableDataClass("client", "datev")

tmp_table, table

(TableDataClass(table_name='tmp_client', database='datev', schema=None, pk=None),
 TableDataClass(table_name='client', database='datev', schema=None, pk=None))

In [7]:
spark = hive_client.create_spark_session()
df_tmp_client = spark.sql(f"SELECT * FROM datev.tmp_client")
df_client = spark.sql(f"SELECT * FROM datev.client WHERE is_batch_file = 1")

base_columns = df_client.columns
base_columns_tmp = df_tmp_client.columns

df_tmp_client.show()
df_client.show()

24/01/04 18:07:32 WARN Utils: Your hostname, MacBook-Air-von-Enrico.local resolves to a loopback address: 127.0.0.1; using 192.168.0.4 instead (on interface en0)
24/01/04 18:07:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/04 18:07:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `datev`.`tmp_client` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 1 pos 14;
'Project [*]
+- 'UnresolvedRelation [datev, tmp_client], [], false


In [16]:
df_tmp_client.join(df_client, on="pk").show()

+---+--------------------+---+--------------------+--------------------+------------+--------------+-------------------+--------------+--------------------+--------------------+-------------+--------------------+---+--------------------+--------------------+------------+--------------+-------------------+--------------+--------------------+--------------------+-------------+
| pk|                guid| id|                name|             address|change_field|row_is_current|       row_valid_to|row_is_deleted|        row_filepath|    row_filepath_tmp|is_batch_file|                guid| id|                name|             address|change_field|row_is_current|       row_valid_to|row_is_deleted|        row_filepath|    row_filepath_tmp|is_batch_file|
+---+--------------------+---+--------------------+--------------------+------------+--------------+-------------------+--------------+--------------------+--------------------+-------------+--------------------+---+--------------------+-------

In [17]:
from pyspark.sql import functions as f

# update dummy
df_tmp_client = df_tmp_client.withColumn("name", f.when(f.col("pk") == 91, "here_name_changed").otherwise(f.col("name")))
df_tmp_client = df_tmp_client.withColumn("name", f.when(f.col("pk") == 92, "here_name_changed").otherwise(f.col("name")))
df_tmp_client = df_tmp_client.withColumn("name", f.when(f.col("pk") == 57, "here_name_changed").otherwise(f.col("name")))

# delete dummy
df_tmp_client = df_tmp_client.filter(f.col("pk") != 30)
df_tmp_client = df_tmp_client.filter(f.col("pk") != 66)
df_tmp_client = df_tmp_client.filter(f.col("pk") != 17)

In [60]:

# transformations

df_client = df_client.withColumn("CHANGE_KEY", f.concat_ws("_", *["name", "change_field"]))
df_tmp_client_ = df_tmp_client.withColumn("CHANGE_KEY_TMP", f.concat_ws("_", *["name", "change_field"]))

df_tmp_client_ = df_tmp_client_.withColumnRenamed("guid", "guid_tmp")
df_tmp_client_ = df_tmp_client_.withColumnRenamed("row_filepath", "row_new_filepath")
df_tmp_client_ = df_tmp_client_.withColumnRenamed("row_filepath_tmp", "row_new_tmp_filepath")

df_tmp_client_ = df_tmp_client_.select(*["pk", "guid_tmp", "row_new_filepath", "row_new_tmp_filepath", "CHANGE_KEY_TMP"])
df_joined = df_client.join(df_tmp_client_, on="pk", how="left")




df_updated_rows = df_joined.filter(f.col("CHANGE_KEY") != f.col("CHANGE_KEY_TMP"))
df_deleted_rows = df_joined.filter(f.col("CHANGE_KEY_TMP").isNull())


df_updated_rows_pd = pd.DataFrame(
    df_updated_rows.collect(),
    columns=df_updated_rows.columns
)
df_deleted_rows_pd = pd.DataFrame(
    df_deleted_rows.collect(),
    columns=df_deleted_rows.columns
)
df_deleted_rows_pd.loc[:, "row_is_deleted"] = 1

df_changed_rows_pd = pd.concat([df_deleted_rows_pd, df_updated_rows_pd])

df_updated_rows_pd.loc[:, "row_is_current"] = 0
df_updated_rows_pd.loc[:, "row_valid_to"] = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
df_updated_rows_pd.loc[:, "row_filepath"] = df_updated_rows_pd["row_new_filepath"]
df_updated_rows_pd.loc[:, "row_filepath_tmp"] = df_updated_rows_pd["row_new_tmp_filepath"]

updated_rows_dist_filepaths = list(df_updated_rows_pd["row_new_tmp_filepath"].unique())


path_tmp = "/edw/hive/tmp/datev/dbo/client"
path_psa = "/edw/hive/psa/datev/dbo/client"
all_files = hdfs_client.client.list(path_psa)
update_files = [file_name for file_name in all_files if "BATCH" in file_name]
update_files_sorted = sorted(update_files, reverse=True)

if len(update_files_sorted) > 0:
    last_update_file: str = update_files_sorted[0]
    last_number = int(last_update_file.rsplit(".", 1)[0].split("_")[1])

    last_update_filepath = "/".join([path_tmp, last_update_file])

    df_last_delete_file = hdfs_client.read_parquet(last_update_filepath)

    write_path = f"{path_tmp}/{last_update_file}"
    if df_last_delete_file.shape[0] > 10_000:
        new_update_filename = f"UPDATE_{last_number + 1}.parquet"
        write_path = f"{path_tmp}/{new_update_filename}"

    data_content = df_updated_rows_pd.to_parquet(index=False, compression="snappy")
    # hdfs_client.write(
    #     data=data_content,
    #     hdfs_path=write_path,
    #     append=True
    # )
    print(write_path, "\n")
else:
    write_path = f"{path_tmp}/UPDATE_0.parquet"
   
    data_content = df_updated_rows_pd.to_parquet(index=False, compression="snappy")
    print(write_path, "\n")
    # hdfs_client.write(
    #     data=data_content,
    #     hdfs_path=write_path,
    #     append=True
    # )

for update_filename in update_files:
    old_filepath = "/".join([path_psa, update_filename])
    new_filepath = "/".join([path_tmp, update_filename])
    print(old_filepath, new_filepath)
    # hdfs_client.client.rename(
    #     hdfs_src_path=old_filepath,
    #     hdfs_dst_path=new_filepath
    # )

    # df_current = hdfs_client.read_parquet(filepath_tmp)
    # df_current = df_current[~df_current["guid"].isin(df_updates["guid_tmp"])]

    # df_updates = df_updates[base_columns]
    # df_transformed = pd.concat([df_updates, df_current], axis=0)

    # display(df_updates, df_current, df_transformed)



    # print(filepath_tmp)
    # display(df_updates)


/edw/hive/tmp/datev/dbo/client/BATCH_3.parquet 

/edw/hive/tmp/datev/dbo/client/BATCH_0.parquet /edw/hive/psa/datev/dbo/client/BATCH_0.parquet
/edw/hive/tmp/datev/dbo/client/BATCH_1.parquet /edw/hive/psa/datev/dbo/client/BATCH_1.parquet
/edw/hive/tmp/datev/dbo/client/BATCH_2.parquet /edw/hive/psa/datev/dbo/client/BATCH_2.parquet
/edw/hive/tmp/datev/dbo/client/BATCH_3.parquet /edw/hive/psa/datev/dbo/client/BATCH_3.parquet


In [19]:

# transformations

df_client = df_client.withColumn("CHANGE_KEY", f.concat_ws("_", *["name", "change_field"]))
df_tmp_client_ = df_tmp_client.withColumn("CHANGE_KEY_TMP", f.concat_ws("_", *["name", "change_field"]))

df_tmp_client_ = df_tmp_client_.withColumnRenamed("guid", "guid_tmp")
df_tmp_client_ = df_tmp_client_.withColumnRenamed("row_filepath", "row_new_filepath")
df_tmp_client_ = df_tmp_client_.withColumnRenamed("row_filepath_tmp", "row_new_tmp_filepath")


df_tmp_client_ = df_tmp_client_.select(*["pk", "guid_tmp", "row_new_filepath", "row_new_tmp_filepath", "CHANGE_KEY_TMP"])
df_joined = df_client.join(df_tmp_client_, on="pk", how="left")




df_updated_rows = df_joined.filter(f.col("CHANGE_KEY") != f.col("CHANGE_KEY_TMP"))
df_deleted_rows = df_joined.filter(f.col("CHANGE_KEY_TMP").isNull())


df_updated_rows_pd = pd.DataFrame(
    df_updated_rows.collect(),
    columns=df_updated_rows.columns
)

df_updated_rows_pd.loc[:, "row_is_current"] = 0
df_updated_rows_pd.loc[:, "row_valid_to"] = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
df_updated_rows_pd.loc[:, "row_filepath"] = df_updated_rows_pd["row_new_filepath"]
df_updated_rows_pd.loc[:, "row_filepath_tmp"] = df_updated_rows_pd["row_new_tmp_filepath"]

updated_rows_dist_filepaths = list(df_updated_rows_pd["row_new_tmp_filepath"].unique())
for filepath_tmp in updated_rows_dist_filepaths:
    df_updates = df_updated_rows_pd[df_updated_rows_pd["row_new_tmp_filepath"] == filepath_tmp]

    df_current = hdfs_client.read_parquet(filepath_tmp)
    df_current = df_current[~df_current["guid"].isin(df_updates["guid_tmp"])]

    df_updates = df_updates[base_columns]
    df_transformed = pd.concat([df_updates, df_current], axis=0)

    display(df_updates, df_current, df_transformed)
    # data_content = df_transformed.to_parquet(index=False, compression="snappy")
    # hdfs_client.write(
    #     data=data_content,
    #     hdfs_path=filepath
    # )


    print(filepath_tmp)
    display(df_updates)
#display(df_updated_rows_pd)

# df_deleted_rows_pd = pd.DataFrame(
#     df_deleted_rows.collect(),
#     columns=df_deleted_rows.columns
# )
# display(df_deleted_rows_pd) ==> hier mit write arbeiten -> einfach alles reinschreiben! SPARK!
    # NEIN => DELETE ALS DELETE FILES SPEICHERN UND FILTERN -> DIESE EINFACH MIT ÜBERNEHMEN

# READ FILE
# TRANSFORM IT
#   - change and add old dataset ==> need GUID & FILEPATH from old dataset
#   => result_df = df_test.groupby('path')['guid'].agg(list).to_dict()
# WRITE IT BACK (override!)
# ==> MIT PANDAS!

# print("CHANGED COLUMNS:")
# df_updated_rows.show()
# df_deleted_rows.show()

hdfs_client.read_parquet("/edw/hive/tmp/datev/dbo/client/BATCH2.parquet")
print("check")

# UND --> wenn filename doppelt! nur einmal öffnen und verändern!

# UPDATE UND DELETE FASSE ICH ABER AUCH NUR EINMAL AN!

,guid,pk,id,name,address,change_field,row_is_current,row_valid_to,row_is_deleted,row_filepath,row_filepath_tmp,is_batch_file
0,70466930-83c5-4820-9090-ff2399883d63,92,92,Xcel Ventures AG,"Musterstraße 92, 10198 Berlin",cf,0,2024-01-04 00:00:00,0,/edw/hive/psa/datev/dbo/client/client/BATCH_2....,/edw/hive/tmp/datev/dbo/client/BATCH_2.parquet,1
1,15464f6f-402d-4922-b575-28138a6b7751,91,91,Wavelength Ventures AG,"Musterstraße 91, 10197 Berlin",cf,0,2024-01-04 00:00:00,0,/edw/hive/psa/datev/dbo/client/client/BATCH_2....,/edw/hive/tmp/datev/dbo/client/BATCH_2.parquet,1


,id,name,address,change_field,guid,pk,row_valid_to,row_is_current,row_is_deleted,row_filepath,row_filepath_tmp,is_batch_file
0,49,Momentum Innovations SE,"Musterstraße 49, 10155 Berlin",cf,56fb3c9f-e7f4-4712-980d-12a1a3115265,49,2100-12-31 00:00:00,1,0,/edw/hive/psa/datev/dbo/client/client/BATCH_2....,/edw/hive/tmp/datev/dbo/client/BATCH_2.parquet,1


,guid,pk,id,name,address,change_field,row_is_current,row_valid_to,row_is_deleted,row_filepath,row_filepath_tmp,is_batch_file
0,70466930-83c5-4820-9090-ff2399883d63,92,92,Xcel Ventures AG,"Musterstraße 92, 10198 Berlin",cf,0,2024-01-04 00:00:00,0,/edw/hive/psa/datev/dbo/client/client/BATCH_2....,/edw/hive/tmp/datev/dbo/client/BATCH_2.parquet,1
1,15464f6f-402d-4922-b575-28138a6b7751,91,91,Wavelength Ventures AG,"Musterstraße 91, 10197 Berlin",cf,0,2024-01-04 00:00:00,0,/edw/hive/psa/datev/dbo/client/client/BATCH_2....,/edw/hive/tmp/datev/dbo/client/BATCH_2.parquet,1
0,56fb3c9f-e7f4-4712-980d-12a1a3115265,49,49,Momentum Innovations SE,"Musterstraße 49, 10155 Berlin",cf,1,2100-12-31 00:00:00,0,/edw/hive/psa/datev/dbo/client/client/BATCH_2....,/edw/hive/tmp/datev/dbo/client/BATCH_2.parquet,1


/edw/hive/tmp/datev/dbo/client/BATCH_2.parquet


,guid,pk,id,name,address,change_field,row_is_current,row_valid_to,row_is_deleted,row_filepath,row_filepath_tmp,is_batch_file
0,70466930-83c5-4820-9090-ff2399883d63,92,92,Xcel Ventures AG,"Musterstraße 92, 10198 Berlin",cf,0,2024-01-04 00:00:00,0,/edw/hive/psa/datev/dbo/client/client/BATCH_2....,/edw/hive/tmp/datev/dbo/client/BATCH_2.parquet,1
1,15464f6f-402d-4922-b575-28138a6b7751,91,91,Wavelength Ventures AG,"Musterstraße 91, 10197 Berlin",cf,0,2024-01-04 00:00:00,0,/edw/hive/psa/datev/dbo/client/client/BATCH_2....,/edw/hive/tmp/datev/dbo/client/BATCH_2.parquet,1


...

,id,name,address,change_field,guid,pk,row_valid_to,row_is_current,row_is_deleted,row_filepath,row_filepath_tmp,is_batch_file
1,19,Epic Solutions SE,"Musterstraße 19, 10125 Berlin",cf,e6bbb303-dcf2-409f-bde5-410c5dc1ec23,19,2100-12-31 00:00:00,1,0,/edw/hive/psa/datev/dbo/client/client/BATCH_0....,/edw/hive/tmp/datev/dbo/client/BATCH_0.parquet,1
2,66,Quantum Consulting SE,"Musterstraße 66, 10172 Berlin",cf,d73b1652-55e3-40a7-a4f9-34a1b8c53fc0,66,2100-12-31 00:00:00,1,0,/edw/hive/psa/datev/dbo/client/client/BATCH_0....,/edw/hive/tmp/datev/dbo/client/BATCH_0.parquet,1


,guid,pk,id,name,address,change_field,row_is_current,row_valid_to,row_is_deleted,row_filepath,row_filepath_tmp,is_batch_file
2,805656fe-4ef7-48e5-8fb5-4b50bc45a73d,57,57,Omicron Systems GmbH,"Musterstraße 57, 10163 Berlin",cf,0,2024-01-04 00:00:00,0,/edw/hive/psa/datev/dbo/client/client/BATCH_0....,/edw/hive/tmp/datev/dbo/client/BATCH_0.parquet,1
1,e6bbb303-dcf2-409f-bde5-410c5dc1ec23,19,19,Epic Solutions SE,"Musterstraße 19, 10125 Berlin",cf,1,2100-12-31 00:00:00,0,/edw/hive/psa/datev/dbo/client/client/BATCH_0....,/edw/hive/tmp/datev/dbo/client/BATCH_0.parquet,1
2,d73b1652-55e3-40a7-a4f9-34a1b8c53fc0,66,66,Quantum Consulting SE,"Musterstraße 66, 10172 Berlin",cf,1,2100-12-31 00:00:00,0,/edw/hive/psa/datev/dbo/client/client/BATCH_0....,/edw/hive/tmp/datev/dbo/client/BATCH_0.parquet,1


/edw/hive/tmp/datev/dbo/client/BATCH_0.parquet


,guid,pk,id,name,address,change_field,row_is_current,row_valid_to,row_is_deleted,row_filepath,row_filepath_tmp,is_batch_file
2,805656fe-4ef7-48e5-8fb5-4b50bc45a73d,57,57,Omicron Systems GmbH,"Musterstraße 57, 10163 Berlin",cf,0,2024-01-04 00:00:00,0,/edw/hive/psa/datev/dbo/client/client/BATCH_0....,/edw/hive/tmp/datev/dbo/client/BATCH_0.parquet,1


HdfsError: File /edw/hive/tmp/datev/dbo/client/BATCH2.parquet not found.

In [ ]:
df_updated_rows_pd = pd.DataFrame(df_updated_rows.collect(), columns=df_updated_rows.columns)
df_updated_rows_pd

,pk,guid,id,name,address,change_field,row_is_current,row_valid_to,row_is_deleted,row_filepath,row_filepath_tmp,is_batch_file,CHANGE_KEY,guid_tmp,row_new_filepath,row_new_tmp_filepath,CHANGE_KEY_TMP
0,92,25fe0eb6-16f2-4d7b-b68e-4d93aa7a65f9,92,Xcel Ventures AG,"Musterstraße 92, 10198 Berlin",cf,1,2100-12-31 00:00:00,0,/edw/hive/psa/datev/dbo/client/client/BATCH2.p...,/edw/hive/tmp/datev/dbo/client/BATCH2.parquet,1,Xcel Ventures AG_cf,aa3e617e-b24e-41c9-9dd7-da339a3311c6,/edw/hive/psa/datev/dbo/client/client/BATCH2.p...,/edw/hive/tmp/datev/dbo/client/BATCH2.parquet,here_name_changed_cf
1,91,932cd61c-a4a4-48e9-94c9-c43e0d5eca3d,91,Wavelength Ventures AG,"Musterstraße 91, 10197 Berlin",cf,1,2100-12-31 00:00:00,0,/edw/hive/psa/datev/dbo/client/client/BATCH2.p...,/edw/hive/tmp/datev/dbo/client/BATCH2.parquet,1,Wavelength Ventures AG_cf,6906d921-35de-4bc2-989f-27f425c42c34,/edw/hive/psa/datev/dbo/client/client/BATCH2.p...,/edw/hive/tmp/datev/dbo/client/BATCH2.parquet,here_name_changed_cf
2,57,af764816-3856-4ec0-9c01-36b06f7a8572,57,Omicron Systems GmbH,"Musterstraße 57, 10163 Berlin",cf,1,2100-12-31 00:00:00,0,/edw/hive/psa/datev/dbo/client/client/BATCH0.p...,/edw/hive/tmp/datev/dbo/client/BATCH0.parquet,1,Omicron Systems GmbH_cf,b46d271e-bb19-4012-a473-e0603567f512,/edw/hive/psa/datev/dbo/client/client/BATCH0.p...,/edw/hive/tmp/datev/dbo/client/BATCH0.parquet,here_name_changed_cf


In [ ]:
df_updated_rows_pd

,pk,guid,id,name,address,change_field,row_is_current,row_valid_to,row_is_deleted,row_filepath,row_filepath_tmp,is_batch_file,CHANGE_KEY,guid_tmp,row_new_filepath,row_new_tmp_filepath,CHANGE_KEY_TMP
0,92,25fe0eb6-16f2-4d7b-b68e-4d93aa7a65f9,92,Xcel Ventures AG,"Musterstraße 92, 10198 Berlin",cf,1,2100-12-31 00:00:00,0,/edw/hive/psa/datev/dbo/client/client/BATCH2.p...,/edw/hive/tmp/datev/dbo/client/BATCH2.parquet,1,Xcel Ventures AG_cf,aa3e617e-b24e-41c9-9dd7-da339a3311c6,/edw/hive/psa/datev/dbo/client/client/BATCH2.p...,/edw/hive/tmp/datev/dbo/client/BATCH2.parquet,here_name_changed_cf
1,91,932cd61c-a4a4-48e9-94c9-c43e0d5eca3d,91,Wavelength Ventures AG,"Musterstraße 91, 10197 Berlin",cf,1,2100-12-31 00:00:00,0,/edw/hive/psa/datev/dbo/client/client/BATCH2.p...,/edw/hive/tmp/datev/dbo/client/BATCH2.parquet,1,Wavelength Ventures AG_cf,6906d921-35de-4bc2-989f-27f425c42c34,/edw/hive/psa/datev/dbo/client/client/BATCH2.p...,/edw/hive/tmp/datev/dbo/client/BATCH2.parquet,here_name_changed_cf
2,57,af764816-3856-4ec0-9c01-36b06f7a8572,57,Omicron Systems GmbH,"Musterstraße 57, 10163 Berlin",cf,1,2100-12-31 00:00:00,0,/edw/hive/psa/datev/dbo/client/client/BATCH0.p...,/edw/hive/tmp/datev/dbo/client/BATCH0.parquet,1,Omicron Systems GmbH_cf,b46d271e-bb19-4012-a473-e0603567f512,/edw/hive/psa/datev/dbo/client/client/BATCH0.p...,/edw/hive/tmp/datev/dbo/client/BATCH0.parquet,here_name_changed_cf


In [ ]:
hdfs_client.client.status("/edw/hive/tmp/")

{'accessTime': 0,
 'blockSize': 0,
 'childrenNum': 1,
 'fileId': 40344,
 'group': 'supergroup',
 'length': 0,
 'modificationTime': 1704281132017,
 'owner': 'enricogoerlitz',
 'pathSuffix': '',
 'permission': '755',
 'replication': 0,
 'storagePolicy': 0,
 'type': 'DIRECTORY'}

In [ ]:
parquet_file_path = '/edw/hive/tmp/datev/dbo/client/BATCH8.parquet'

hdfs_client.read_parquet(parquet_file_path)

HdfsError: File /edw/hive/tmp/datev/dbo/client/BATCH8.parquet not found.

In [ ]:
import pyarrow.parquet as pq
from io import BytesIO

parquet_file_path = '/edw/hive/tmp/datev/dbo/client/BATCH8.parquet'
with hdfs_client.client.read(parquet_file_path) as f:
    display(pd.read_parquet(BytesIO(f.read())))
    # print(f.read())
    #print(f.read())

HdfsError: File /edw/hive/tmp/datev/dbo/client/BATCH8.parquet not found.

In [ ]:
df_test = pd.DataFrame({
    "path": ["/file/path/1", "/file/path/1", "/file/path/1", "/file/path/2", "/file/path/2", "/file/path/3"],
    "guid": [20, 89, 10, 11, 90, 101]
})
df_test

,path,guid
0,/file/path/1,20
1,/file/path/1,89
2,/file/path/1,10
3,/file/path/2,11
4,/file/path/2,90
5,/file/path/3,101


In [ ]:
result_df = df_test.groupby('path')['guid'].agg(list).to_dict()

# Umbenennen des Index in 'path'
result_df

{'/file/path/1': [20, 89, 10], '/file/path/2': [11, 90], '/file/path/3': [101]}